### 데이터 로드

In [1]:
import pandas as pd

%load_ext google.colab.data_table

data = pd.read_csv('/content/drive/MyDrive/socar data/memo_hackathon_lesion_info_table.tsv')

# data[:20000]

### 전체데이터에서 필요한 칼럼만 추출

In [2]:
# data.columns

In [3]:
cols = ['lesion_id', 'lesion_detail_type', 'memo_id', 'memo', 'inspect_id', 'inspect_type', 'price', 'description']
data_part = data[cols]

# data_part[:20000]

'memo_id' 중복 제거

In [4]:
data_part.drop_duplicates('memo_id', inplace=True)
data_part.dropna(axis=0, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
# data_part    # 50952개

## memo 데이터 전처리

- 양식이 매번 업데이트 되므로 거의 양식이 없다고 봐도 무방
- 우선 양식이 있는 데이터에서 필요한 정보만 추려보자.
- memo데이터를 문장별로 구분할것인지 말것인지
- 구분한다면 어떤 형태로 저장할 것인지

### 신규접수, 처리중, 조치완료, 검수중 과 같은 시스템이 자동입력한 데이터 제거

In [6]:
import re

#pattern = r'(신규접수->처리중)|(처리중->조치완료)|(조치완료->검수중) \([0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}\)'    # 이처럼 날짜 형식을 추가한 것과 제거한 것으로 찾은 데이터수는 같다. (36781개)
pattern = r'(신규접수->처리중)|(처리중->조치완료)|(처리중->조치불필요)|(처리중->조치불가)|(조치완료->검수중)|(조치완료->검수중)|(검수중->검수완료)'

# data_part[ data_part['memo'].apply(lambda s: bool(re.search(pattern, str(s))))]    # 20139개

실제로 내가 기대한 텍스트를 탐색한 것인지 확인해본다.
- OK

In [7]:
test_str = data_part['memo'].to_string()
# print(re.findall(pattern, test_str))

drop후 새로운 dataframe에 저장한다.

In [8]:
df_idx = data_part[ data_part['memo'].apply(lambda s: bool(re.search(pattern, str(s))))].index
data_part_preprocessing = data_part.drop(df_idx)

# data_part_preprocessing    # 30813개

중복제거한 데이터 50952개에서 불필요한 memo데이터가 20139개가 제거되어, 30813개가 남았다.

### 한글 맞춤법 검사



In [9]:
!pip install git+https://github.com/ssut/py-hanspell.git # 띄어쓰기/ 맞춤법 교정

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-8zp4a4qj
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-8zp4a4qj
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=b8d72d1112442b6e179556fa5cc62f989ad6b1adc2b6ec80381add0ce9106180
  Stored in directory: /tmp/pip-ephem-wheel-cache-kdtmroub/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

In [11]:
count = 0

#for row in data_part_preprocessing.values:
for i,row in data_part_preprocessing.iterrows():
  # print(i)
  # print(row['memo'])
  
  # memo = row['memo'].replace('\n\n', '\n')
  # memo_sent = memo.split('\n')
  # print(memo_sent)
  #print(correction(row['memo']))
  if count==10:
    break
  count+=1

In [ ]:
import re

count = 0

for i,row in data_part_preprocessing.iterrows():
  #print(row['memo'])

  memo = row['memo']
  content = re.findall('(ㅁ문의내용[^ㅁ]+ㅁ)|(ㅁ 문의내용[^ㅁ]+ㅁ)|([^ㅁ ]+문의내용[^ㅁ]+ㅁ)', memo)

  if content:
    if content[0][2]:
      print(content)
  # if content:
  #   if content[0][0]:
  #     print(content[0][0])
  #   elif content[0][1]:
  #     print(content[0][1])
  #   elif content[0][2]:
  #     print(content[0][2])
    


  if count==10000:
    break
  count+=1

### extract_info함수

In [13]:
# data의 'memo'에 NaN이 있으면 오류발생함.
def extract_info(data):
  result_df = pd.DataFrame(columns=['lesion_id', 'memo_info'])

  # 테스트 시)
  # count = 0
  for i,row in data.iterrows():
    memo = row['memo']
    memo = re.sub('문의사항', '문의내용', memo)
    content = re.findall('(ㅁ문의[ ]?내용[\s\S]+ㅁ확인/처리 내용)|(ㅁ문의[ ]?내용[^ㅁ]+ㅁ)|(ㅁ 문의[ ]?내용[^ㅁ]+ㅁ)|([^ㅁ ]+문의[ ]?내용[^ㅁ]+ㅁ)', memo)
    content2 = re.findall('◎[ ]?문의[ ]?내용[^◎]+◎', memo)

    # if row['lesion_id']==697976:
    #   print(memo)
    #   print('content: ')
    #   print(content)

    # 문의내용이 있는 경우
    text = ''
    if content2:
      text = content2[0]
    
    else:
      if content:
        # print(content)
        if content[0][0]:
          text = content[0][0]
        elif content[0][1]:
          text = content[0][1]
        elif content[0][2]:
          text = content[0][2]
    
    if text:
      # 의미없는 텍스트 제거
      text = re.sub(r'문의내용', ' ', text, flags=re.MULTILINE)
      text = re.sub(r'안내/처리 내용', ' ', text, flags=re.MULTILINE)
      text = re.sub(r'확인/처리 내용', ' ', text, flags=re.MULTILINE)
      text = re.sub(r'확인내용', ' ', text, flags=re.MULTILINE)
      #text = re.sub(r'인입', ' ', text, flags=re.MULTILINE)
      text = re.sub(r'010-0000-0000', ' ', text, flags=re.MULTILINE)

      text = re.sub(r'[\nㅁㄴ＊*◎:/,;\-\.]', ' ', text, flags=re.MULTILINE)
      text = re.sub(r'[ ]+', ' ', text)

      if (text.split(' ')[0] == '완료') | (text.split(' ')[0] == '미완료'):
        text = text[3:]
      
      text = text.strip()
      
      # 여기에 중심단어로 replace하는 작업 필요
      # 아니면 토큰화 후 replace하는 게 더 나을수도 있겠다.
      if len(text) > 1:
        result_df = result_df.append(pd.Series([int(row['lesion_id']), text], index=result_df.columns), ignore_index=True)
    
    # 테스트 시)
    # if count==1000:
    #   break
    # count+=1

  
  return result_df

    
df = extract_info(data_part_preprocessing)

In [14]:
# df

In [15]:
# df.to_csv('/content/drive/MyDrive/socar data/memo_info.csv')

In [16]:
df['lesion_id'].nunique()

6855

In [17]:
data['lesion_id'].nunique()

9477

결과: 전체 lesion_id 9477개 중 6848개에 대한 memo 데이터로부터 유의미한 정보를 추출하였다.

다음은 'memo'가 NaN이어서 오류발생했던 행이다.

In [18]:
# data[data['lesion_id']==688285]

In [19]:
lesion_id = list(df['lesion_id'])

In [20]:
data_part_preprocessing['lesion_id'].nunique()    # data와 lesion_id 수가 다르다. (이 56개 id에 대한 memo는 무엇일까)

9421

In [21]:
data_else = data_part_preprocessing[data_part_preprocessing['lesion_id'].isin(lesion_id) == False]

# data_else

남은 데이터들의 양식을 살펴보자.

- \# 사고조사
- \# 사고조사_요청
- \# 조치완료
- \# 문제어불가
- \# 네비게이션
- 차량번호/차종/숫자/내용 4자리숫자/중증
- 가온오토)


어떻게 처리할까?
- 타이어규격 제거 `[0-9]{3}/[0-9]{2}/[0-9]{2}`
- 날짜 제거 `[0-9]{2}/[0-9]{2}`
- 차량번호 제거 `[0-9]{2}[가-힣][0-9]{4}` 또는 `[0-9]{3}[가-힣][0-9]{4}`
- '가온오토)' 제거
- '010-0000-0000' 제거
- `원예약번호 : [0-9]{8}`으로 re.search한 memo들은 따로 처리
 - 증상 상세내용 항목에 있는 텍스트 추출
 - 최대한 항목에 있는 텍스트를 추출 후, 남은 데이터들은 원예약번호를 제거한 나머지 텍스트를 그대로 가져온다. 대부분 '/'로 구분되어 있음. 특수문자 제거 필요.
- '문의내용'이나 '문의사항'을 포함하고 있는 memo데이터는 이로부터 유의미한 정보를 추출하지 못했으므로 다른 항목에서 정보를 추출해야 한다.
- '[긴급출동 접수]'에서는 '접수내용'과 '접수 상세 사유'라는 항목에서 정보를 추출한다.

- 'lesion_id' 669141데이터는 제거한다. 

In [ ]:
def extract_info_else(data):
  result_df = pd.DataFrame(columns=['lesion_id', 'memo_info'])

  count=0
  for i,row in data.iterrows():
    memo = row['memo']
    memo = re.sub('[0-9]{2,3}[가-힣][ ]*[0-9]{4}', ' ', memo)    # 차량번호 제거
    memo = re.sub('\(\d{2,4}/\d{1,2}/\d{1,2}\)', ' ', memo)      # 소괄호 포함한 타이어 규격 제거
    memo = re.sub('\d{2,4}/\d{1,2}/\d{1,2}', ' ', memo)          # 타이어 규격 또는 날짜(연/월/일) 제거
    memo = re.sub('\d{1,2}/\d{1,2}\s', ' ', memo)                # 날짜(월/일) 제거
    memo = re.sub('\(.*\)', ' ', memo)                           # 소괄호와 소괄호 안에 있는 내용 제거
    memo = re.sub('010\-0000\-0000', ' ', memo)

    content1 = re.findall(r'\[긴급출동 접수\]', memo)    # 긴급출동
    content2 = re.findall(r'#[ ]*[^\n]+', memo)          # 태그가 붙은 데이터
    content3 = re.findall(r'증상 상세내용[^ㅁ]+', memo)

    #content2 = re.findall('(\w+\/\w+\/\w+\/[\w ]+\/[\w ]+)|([0-9]{3}/[0-9]{2}/[0-9]{2})|([0-9]{2,3}[가-힣][0-9]{4})|(# [가-힣]{4,6})', memo)

    # 긴급출동 접수 (132개)
    if content1:
      # print(re.findall('(\- 접수내용[ ]*:[ ]*[^\-]+)|(\- 접수 상세 사유[ ]*:[ ]*[^\-]+)', memo))
      info = re.findall('(\- 접수내용[ ]*:[ ]*[^\-]+)|(\- 접수 상세 사유[ ]*:[ ]*[^\-]+)', memo)
      text1 = re.sub('\- 접수내용[ ]*:[ ]*', '', info[0][0])
      text1 = text1.strip()
      text2 = re.sub('\- 접수 상세 사유[ ]*:[ ]*', '', info[1][1])
      text2 = text2.strip()
    
    elif content2:
      # print()
      # print(row['lesion_id'])
      # print(content2)
      for c in content2:
        if bool(re.search(r'사고조사|선제조치|조치완료|차량상태확인', c)):
          memo = re.sub(c, ' ', memo)
        else:
          memo = re.sub('#', '', memo)
      
      # print(memo)
      count+=1
        
      # result_df = result_df.append(pd.Series([int(row['lesion_id']), text1+' '+text2], index=result_df.columns), ignore_index=True)
    
    elif content3:
      print()
      print(row['lesion_id'])
      print(content3)
      info = re.sub('증상\s*상세\s*내용\s*:\s*', ' ', str(content3))
      print(info)
      print(memo)
  print(count)

extract_info_else(data_else)

In [23]:
a = '# 사고조치완료'
b= r'사고조사|선제조치|조치완료|차량상태확인'

bool(re.search(b, a))

True

### '문의내용' 항목 정보 추출 관련 여러가지 테스트

memo데이터 중 많은 부분을 차지하는 몇가지 양식이 있다.

**양식1: '$고객'으로 시작**
- 문의내용
- 확인내용
- 안내/처리내용
- 약속 내용

**양식2: '원예약번호'로 시작**
- 원예약번호
- 증상 상세내용
- 문의내용
- 확인내용
- 안내/처리 내용
- 약속 내용

또는
- 경고등 사진 여부
- 타이어 펑크
등 type별로 조금씩 달라짐.

**양식3: '[긴급출동 접수]'로 시작**
- 연락가능한번호
- 고객과실여부
- 고객과실사유
- 접수내용
- 접수상세사유
- 일반도로 현주소
- 현장 고객 유무



**양식4: '1. 분실여부'로 시작**

**양식5: '1. 증상 상세내용'으로 시작**

In [24]:
# data_part_preprocessing[:20000]

In [ ]:
pattern2 = r'문의내용'
data_part_preprocessing[data_part_preprocessing['memo'].apply(lambda s: bool(re.search(pattern2, str(s))))]    # 7015개

In [ ]:
pattern2 = r'ㅁ문의내용[^ㅁ]+ㅁ'
data_part_preprocessing[data_part_preprocessing['memo'].apply(lambda s: bool(re.search(pattern2, str(s))))]    # 6605개

In [ ]:
pattern2 = r'ㅁ 문의내용[^ㅁ]+ㅁ'    # ㅁ 띄고 문의내용을 적은 경우
data_part_preprocessing[data_part_preprocessing['memo'].apply(lambda s: bool(re.search(pattern2, str(s))))]    # 287개

In [ ]:
pattern2 = r'[^ㅁ ]+문의내용[^ㅁ]+ㅁ'    # ㅁ이 앞에 붙지 않는 경우
data_part_preprocessing[data_part_preprocessing['memo'].apply(lambda s: bool(re.search(pattern2, str(s))))]    # 110개